In [ ]:
!pip install -q -U bitsandbytes
!pip install transformers==4.34.1
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install huggingface_hub
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer

#hf_token = ""  # Replace with your actual token

#model_id = "Abira1/llama-2-7b-finance"
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
#model_id = "meta-llama/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
model.config.use_cache = True
model.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (nor

In [ ]:
messages = [
    {"role": "system", "content": "You are a personal chatbot who always responds in professional tone and explain things clearly!"},
    {"role": "user", "content": "Who are you and what you can do?"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)


terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


I'm delighted to introduce myself! I'm a personal chatbot, designed to assist and provide valuable information to individuals seeking answers or guidance on various topics. My primary objective is to provide clear, concise, and accurate responses to your queries, ensuring that you receive the most relevant and helpful information.

I can assist with a wide range of topics, including but not limited to:

1. General knowledge: I can provide information on various subjects, such as science, history, technology, and more.
2. Language translation: I can translate text from one language to another, helping bridge language barriers.
3. Definitions: I can define technical terms, jargon, and complex concepts, making it easier to understand and communicate effectively.
4. Problem-solving: I can help you brainstorm solutions to problems you're facing, providing suggestions and ideas to tackle challenges.
5. Writing assistance: I can assist with writing tasks, such as proofreading, editing, and su

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer

def stream(user_prompt):
  """
  Generates an essay based on a user-provided prompt using a pre-trained causal language model.

  Args:
      tokenizer (AutoTokenizer): The tokenizer for the chosen model.
      model (AutoModelForCausalLM): The pre-trained causal language model.
      runtime_flag (str, optional): The device to run the model on (e.g., "cuda:0" for GPU). Defaults to "cuda:0".

  Returns:
      str: The generated essay based on the user prompt.
  """

  runtime_flag="cuda:0"

  # Create TextStreamer for efficient memory handling during generation
  text_streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

  # Prepare input for the model
  input_ids = tokenizer.apply_chat_template(
      [{"role": "user", "content": user_prompt}], add_generation_prompt=True, return_tensors="pt"
  ).to(runtime_flag)

  # Define termination conditions
  terminators = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]

  # Generate text using the model and streamer
  outputs = model.generate(
      input_ids,
      max_new_tokens=2056,  # Adjust as needed
      eos_token_id=terminators,
      do_sample=True,
      streamer=text_streamer
  )

  # Decode the generated tokens into text
  generated_text = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

  return generated_text

# Example usage (assuming you have a trained tokenizer and model)
essay = stream("Who are you")
print("This is print Statement",essay)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


I am LLaMA, an AI assistant developed by Meta AI that can understand and respond to human input in a conversational manner. I'm not a human, but a computer program designed to simulate conversation, answer questions, and even generate text or code. My primary goal is to assist and communicate effectively with users like you.

I was trained on a massive dataset of text from the internet and can generate human-like responses to a wide range of topics and questions. I can be used for a variety of tasks, such as:

1. Answering questions: I can provide information on a vast range of topics, from science and history to entertainment and culture.
2. Generating text: I can create original text based on a prompt, topic, or style.
3. Translation: I can translate text from one language to another.
4. Summarization: I can summarize long pieces of text into shorter, more digestible formats.
5. Conversation: I can engage in natural-sounding conversations, using context and understanding to respond t

In [ ]:
# if UTF-8 error occurs run this code
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
#Installing Streamlit app

In [ ]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:00


In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.125.202.255


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.202.255:8501

npx: installed 22 in 2.42s
your url is: https://pink-mugs-suffer.loca.lt
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Loading checkpoint shards: 100% 4/4 [01:19<00:00, 19.76s/it]
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1421: UserWarning: You have mod